In [1]:
import platform
print(platform.architecture())

('64bit', 'WindowsPE')


In [2]:
!python --version

Python 3.6.12 :: Anaconda, Inc.


In [3]:
# 네이버에서 검색어 입력받아 검색 한 후 블로그 메뉴를 선택하고
# 오른쪽에 있는 검색옵션 버튼을 눌러서
# 정렬 방식과 기간을 입력하기

#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys # 시스템
import os # 시스템
import pandas as pd # 데이터 분석
import numpy as np # 숫자

from bs4 import BeautifulSoup # 데이터 파싱
from selenium import webdriver # 브라우저 자동화
import time # 서버응답시간 대기
from tqdm import tqdm_notebook # for문 진행상황 시각화

In [4]:
query_txt = input('1.크롤링할 키워드는 무엇입니까?: ')
# query_txt2 = input('2.제외할 첫번째 키워드를 입력하세요: ')
# query_txt3 = input('3.제외할 두번째 키워드를 입력하세요: ')

start_date = input('4.조회를 시작할 날짜를 입력하세요(예:20170101) :')
end_date = input('5.조회를 종료할 날짜를 입력하세요(예:20171231): ')

1.크롤링할 키워드는 무엇입니까?: 떡국
4.조회를 시작할 날짜를 입력하세요(예:20170101) :20210101
5.조회를 종료할 날짜를 입력하세요(예:20171231): 20210105


In [45]:
query_txt="김치볶음밥 맛집"
start_date="20201201"
end_date="20210106"

# 크롤링할 블로그 url 수집하기

In [5]:
#Step 1. 크롬 웹브라우저 실행
path = "chromedriver.exe"  # 맥 기준  # 윈도우는 "chromedriver.exe"

driver = webdriver.Chrome(path)
# 사이트 주소는 네이버
driver.get('http://www.naver.com')
time.sleep(2)

#Step 2. 네이버 검색창에 "검색어" 검색
element = driver.find_element_by_id("query")
element.send_keys(query_txt)  # query_txt는 위에서 입력한 키워드
element.submit()
time.sleep(1)

# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click( )

# '블로그' 클릭
driver.find_element_by_link_text("블로그").click( )
time.sleep(1)

# '검색옵션' 클릭
driver.find_element_by_link_text("검색옵션").click( )
time.sleep(1)

# Step 3. 정렬 : "관련도순" 
# 개발자 도구에서 정렬 버튼의 id 가 보이지 않습니다.
# 이럴 경우 쉽게 사용할 수 있는 방법이 바로 xpath 를 이용하는 방법입니다.
# xpath는 개발자 도구에서 해당 메뉴 부분을 마우스 오른쪽 버튼을 누르고 
# copy -> copy xpath 를 선택하면 됩니다
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/a""").click( )  # 정렬 버튼의 xpath 클릭
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[1]/a""").click( ) # 관련도순 xpath
time.sleep(1)

# Step 4. 날짜 입력
driver.find_element_by_link_text("기간").click()
time.sleep(2)

# 이 부분이 아주 중요합니다.
# 날짜 부분에 날짜를 입력할 때 입력 속도가 너무 빠를 경우 날짜가 입력이 되다가 
# 오타가 나오는 경우가 많습니다.
# 그래서 날짜를 입력할 때 for 반복문을 사용해서 1 글자씩 입력하도록 코딩했습니다.

# 시작 날짜 입력하기
s_date = driver.find_element_by_id("date_from_input")
driver.find_element_by_id("date_from_input").click()
s_date.clear( )  # 날짜 입력 부분에 기존에 입력되어 있던 날짜를 제거합니다. 
time.sleep(1)
# 아래 코드가 날짜를 for 반복문으로 1 글자씩 입력하는 부분입니다.
for c in start_date:
    s_date.send_keys(c)
    time.sleep(0.1)
time.sleep(1)

# 종료 날짜 입력하기
e_date = driver.find_element_by_id("date_to_input")
driver.find_element_by_id("date_to_input").click()
e_date.clear()
time.sleep(1)

for c in end_date:
    e_date.send_keys(c)
    time.sleep(0.1)
    
# Step5. 날짜 입력 "적용" 버튼을 클릭 합니다.  
driver.find_element_by_class_name("tx").click()
time.sleep(3)

# # Step 8. 상세 검색버튼을 클릭 후 제외할 단어들을 설정합니다.
# ele2 = driver.find_element_by_id("inpop3")
# ele2.send_keys(query_txt2)
# ele2.send_keys(',')
# ele2.send_keys(query_txt3)
# driver.find_element_by_css_selector(".btn_ft.ty_green._search").click( )

## 글 url 크롤링 시작

In [6]:
# 스크롤 다운 

def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 99999999)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 10
i = 0
while i < n:
    scroll_down(driver) # 스크롤 다운
    i += 1

In [7]:
url_list = []
title_list = []

# URL 크롤링 시작
articles = ".api_txt_lines.total_tit"
article_raw = driver.find_elements_by_css_selector(articles)

# url 크롤링 시작    
for article in article_raw:
    url = article.get_attribute('href')   
    url_list.append(url)
time.sleep(1)
    
# 제목 크롤링 시작    
for article in article_raw:
    title = article.text
    title_list.append(title)

    print(title)

print("")
print('url갯수: ', len(url_list))
print('title갯수: ', len(title_list))

속초 한우 진국 설렁탕 진하게 간단 떡국 끓이기
소고기 떡국 끓이는법 사골 떡만두국
떡국 끓이는법 떡만두국 든든하지!
어묵간장조림 만드는 법 떡국떡으로 어묵 떡볶이처럼
떡만두국 끓이는법 멸치육수로 끓이는 맛있는 떡국
2021년 새해 멸치육수 떡국 끓이는 법
소고기 떡국 끓이는법 사골 없이도 깊다
새해맞이 "소고기떡국" 간단하게 끓이기♪
시판 사골육수로 간단하게 떡국 끓이는 법
떡국 끓이는법 지단까지 완벽하게!
간단 떡국 1인분
오늘의요리] 새해떡국..(마켓컬리)
새해맞이 사골 떡국 끓이는 법
알토란 10분완성 소고기떡국 만드는법(feat. 소고기효능)
지금 제철 매생이굴떡국 매생이 간단손질법 알려드릴게요.
백종원 소고기떡국 끓이는법 황금레시피!
제철먹거리 매생이와 굴을 이용해 시원한 매생이굴떡국 끓였어요
소고기떡국 한그릇 드세요.
백파더 백종원 떡국 만드는법
새해엔 꼭 먹어야하는 소고기 떡국 맛있게 끓여서 먹기
식육식탁,퇴근후집밥, 출근전집밥, 떡국, 닭볶음탕,오리불고기...
소고기 떡국 끓이는법 새해 첫날 먹는 음식
소고기떡국 끓이는법 간단한 요리 쉽다
떡국 끓이는법 백파더 소고기떡국 진한 육수 맛!
새해음식 소고기떡국과 소불고기로 준비했어요~ 반값한우
소고기떡국 끓이는법 진하고 깔끔하게
새해떡국 대신에 떡만두국 끓이는법. 멸치육수도 맛나요~
새해아침 떡국 끓이는 법
진하고 맛있는 굴떡국 끓이는법
백종원 소고기떡국 끓이는법 백파더 레시피 계란지단 만들기
떡국떡으로 다양한 요리 해먹자
새해맞이 사골 떡국 끓이는 법
소고기(장)떡국 끓이는법
초간단 떡국 끓이기2
백종원 소고기떡국 끓이는법 백파더 레시피 계란지단 만들기
맛있는 떡만두국 : 떡국 끓이는 법
잠실 시그니엘 스테이 81층 조식 떡국
육수내어 떡국 끓이는법, 쉬운 떡만두국 레시피
새해 떡국만들기 사골국물떡국
초간단 떡국 만들기
[집밥메뉴] 3인밥상 - 떡국
떡국만들기
소고기떡국 한그릇 드세요.
새해 첫날 먹는 떡국 맛있게 끓이는 법
떡국 드시고 새해 복 많이 받으세요. ^^
소고기 떡국 끓이는법 새해 첫

In [8]:
df = pd.DataFrame({'url':url_list, 'title':title_list})

# 저장하기
df.to_excel("blog_url.xlsx")

# 블로그 내용 크롤링하기

In [9]:
import sys
import os
import pandas as pd
import numpy as np

In [10]:
# "url_list.csv" 불러오기
url_load = pd.read_excel("blog_url.xlsx") # 기본 모델

num_list = len(url_load)

print(num_list)
url_load

330


,Unnamed: 0,url,title
0,0,https://blog.naver.com/molynmom?Redirect=Log&l...,속초 한우 진국 설렁탕 진하게 간단 떡국 끓이기
1,1,https://blog.naver.com/beluga11?Redirect=Log&l...,소고기 떡국 끓이는법 사골 떡만두국
2,2,https://blog.naver.com/lovingssu?Redirect=Log&...,떡국 끓이는법 떡만두국 든든하지!
3,3,https://blog.naver.com/naya399?Redirect=Log&lo...,어묵간장조림 만드는 법 떡국떡으로 어묵 떡볶이처럼
4,4,https://blog.naver.com/yee1036?Redirect=Log&lo...,떡만두국 끓이는법 멸치육수로 끓이는 맛있는 떡국
...,...,...,...
325,325,https://blog.naver.com/tndkdl?Redirect=Log&log...,1월1일 떡국 그리고 커피
326,326,https://blog.naver.com/ghdtkaehddl?Redirect=Lo...,소고기떡국 만들기 새해 아침밥 한그릇요리
327,327,https://blog.naver.com/ksilverh80?Redirect=Log...,2021 신축년 첫 날 떡국 밥상
328,328,https://bestar97.blog.me/222192786125,새해 떡국


In [11]:
dict = {}  # 전체 크롤링 데이터를 담을 그릇

# ★수집할 글 갯수 정하기
number = 30
for i in tqdm_notebook(range(0, number)):
    # 글 띄우기
    url = url_load['url'][i]
    driver = webdriver.Chrome("chromedriver.exe")  # 윈도우는 "chromedriver.exe"
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # iframe 접근
        driver.switch_to.frame('mainFrame')

        target_info = {}

        # 제목 크롤링 시작
        overlays = ".se-module.se-module-text.se-title-text"                                 
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text

        # 글쓴이 크롤링 시작
        overlays = ".nick"                                 
        nick = driver.find_element_by_css_selector(overlays)         # nickname
        nickname = nick.text

        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"                                 
        date = driver.find_element_by_css_selector(overlays)         # datetime
        datetime = date.text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)    

        content_list = []
        for content in contents:
            content_list.append(content.text)           
 
        content_str = ' '.join(content_list)                         # content_str

        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        target_info['title'] = title
        target_info['nickname'] = nickname
        target_info['datetime'] = datetime
        target_info['content'] = content_str

        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = target_info
        time.sleep(1)
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)

        # 글 하나 크롤링 후 크롬 창을 닫습니다.
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동합니다.
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기
    if i == 30 or 50 or 80:
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("blog_content.xlsx")
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
print(dict)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/30 [00:00<?, ?it/s]

0 속초 한우 진국 설렁탕 진하게 간단 떡국 끓이기
1 소고기 떡국 끓이는법 사골 떡만두국
2 떡국 끓이는법 떡만두국 든든하지!
3 어묵간장조림 만드는 법 떡국떡으로 어묵 떡볶이처럼
4 떡만두국 끓이는법 멸치육수로 끓이는 맛있는 떡국
5 2021년 새해 멸치육수 떡국 끓이는 법
6 소고기 떡국 끓이는법 사골 없이도 깊다
7 새해맞이 "소고기떡국" 간단하게 끓이기♪
8 시판 사골육수로 간단하게 떡국 끓이는 법
10 간단 떡국 1인분
11 오늘의요리] 새해떡국..(마켓컬리)
12 새해맞이 사골 떡국 끓이는 법
13 알토란 10분완성 소고기떡국 만드는법(feat. 소고기효능)
14 지금 제철  매생이굴떡국  매생이 간단손질법 알려드릴게요.
15 백종원 소고기떡국 끓이는법 황금레시피!
17 소고기떡국 한그릇 드세요.
18 백파더 백종원 떡국 만드는법
19 새해엔 꼭 먹어야하는 소고기 떡국 맛있게 끓여서 먹기
20 식육식탁,퇴근후집밥, 출근전집밥, 떡국, 닭볶음탕,오리불고기,배추된장국,시금치계란말이
21 소고기 떡국 끓이는법 새해 첫날 먹는 음식
22 소고기떡국 끓이는법 간단한 요리 쉽다
23 떡국 끓이는법 백파더 소고기떡국 진한 육수 맛!
24 새해음식 소고기떡국과 소불고기로 준비했어요~ 반값한우
25 소고기떡국 끓이는법 진하고 깔끔하게
26  새해떡국 대신에 떡만두국 끓이는법. 멸치육수도 맛나요~
27 새해아침 떡국 끓이는 법
28 진하고 맛있는 굴떡국 끓이는법
29 백종원 소고기떡국 끓이는법 백파더 레시피 계란지단 만들기
수집한 글 갯수:  28
{0: {'title': '속초 한우 진국 설렁탕 진하게 간단 떡국 끓이기', 'nickname': '초이린\n(molynmom)', 'datetime': '2021. 1. 2. 16:18', 'content': '한해가 시작된지 벌써 이틀이나 지났네요.\n어제는 컨디션이 좋지 않아서 \n끙끙대며 이불 속에서 지낸거 같아요.\n오늘도 사실 그렇게 몸상태가 좋은건 아니에요.\n그래도 이불 싸매고 한숨 자고 나니\n그나마 정

In [12]:
# 판다스로 만들기
import pandas as pd
result_df = pd.DataFrame.from_dict(dict, 'index')

# 엑셀로 저장하기
result_df.to_excel("blog_content.xlsx")